In [1]:
import pickle
import json
import numpy as np
import pandas as pd
from keplergl import KeplerGl
import datetime

In [2]:
# Load NYC GeoJSON and filter to Manhattan only
with open('data/nyc-zip-code-tabulation-areas-polygons.geojson', 'r') as f:
    nyc_geojson = json.load(f)

MANHATTAN_ZIP_CODES = ['10002', '10003', '10005', '10006', '10007', '10009', 
                    '10010', '10011', '10012', '10013', '10014', '10038']

# Filter to only include Manhattan zip codes
manhattan_geojson = {
    'type': 'FeatureCollection',
    'features': [
        feature for feature in nyc_geojson['features']
        if feature['properties'].get('postalCode') in MANHATTAN_ZIP_CODES
    ]
}

print(f"Loaded {len(manhattan_geojson['features'])} Manhattan regions out of {len(nyc_geojson['features'])} total NYC regions")

Loaded 12 Manhattan regions out of 262 total NYC regions


In [3]:
# Extract centroids from Manhattan regions
from shapely.geometry import shape

# Create mapping: region index (0-11) <-> zip code
region_to_zip = {i: zc for i, zc in enumerate(MANHATTAN_ZIP_CODES)}
zip_to_region = {zc: i for i, zc in enumerate(MANHATTAN_ZIP_CODES)}

# Extract centroids for each region
region_centroids = {}
manhattan_features = {}

for feature in manhattan_geojson['features']:
    postal = feature['properties']['postalCode']
    region_idx = zip_to_region[postal]
    manhattan_features[region_idx] = feature
    
    # Compute centroid from geometry
    geom = shape(feature['geometry'])
    centroid = geom.centroid
    region_centroids[region_idx] = (centroid.x, centroid.y)

print(f"Computed centroids for {len(region_centroids)} regions:")
for idx in sorted(region_centroids.keys()):
    lon, lat = region_centroids[idx]
    print(f"  Region {idx} ({region_to_zip[idx]}): ({lon:.4f}, {lat:.4f})")

Computed centroids for 12 regions:
  Region 0 (10002): (-73.9857, 40.7162)
  Region 1 (10003): (-73.9888, 40.7319)
  Region 2 (10005): (-74.0086, 40.7061)
  Region 3 (10006): (-74.0134, 40.7082)
  Region 4 (10007): (-74.0076, 40.7140)
  Region 5 (10009): (-73.9791, 40.7265)
  Region 6 (10010): (-73.9829, 40.7391)
  Region 7 (10011): (-74.0004, 40.7413)
  Region 8 (10012): (-73.9979, 40.7255)
  Region 9 (10013): (-74.0049, 40.7202)
  Region 10 (10014): (-74.0066, 40.7338)
  Region 11 (10038): (-74.0030, 40.7097)


In [4]:
with open('saved_files/reb_flows_mode2_1000cars_nyc_man_south.pkl', 'rb') as f:
    flow_data = pickle.load(f)

In [5]:
# Time settings
BASE_TIME = datetime.datetime(2013, 3, 8, 19, 0, 0)
INTERVAL_MINUTES = 3
NUM_TIMESTEPS = flow_data['agent_reb_flows'][0].shape[0]

def timestep_to_datetime(t):
    """Convert timestep to datetime string."""
    dt = BASE_TIME + datetime.timedelta(minutes=t * INTERVAL_MINUTES)
    return dt.strftime('%Y-%m-%d %H:%M:%S')


def create_flow_df(flow_data, agent_id, region_centroids, edges):
    """Create DataFrame with only NON-ZERO flows for an agent.
    
    Ensures every timestep has at least one row (with tiny flow if needed)
    so time animation doesn't skip timesteps.
    """
    flows = flow_data['agent_reb_flows'][agent_id]
    num_timesteps, num_edges = flows.shape
    
    records = []
    for t in range(num_timesteps):
        timestep_has_flow = False
        for edge_idx, (origin, dest) in enumerate(edges):
            flow_value = flows[t, edge_idx]
            origin_lon, origin_lat = region_centroids[origin]
            dest_lon, dest_lat = region_centroids[dest]
                
            records.append({
                'origin_lon': origin_lon,
                'origin_lat': origin_lat,
                'dest_lon': dest_lon,
                'dest_lat': dest_lat,
                'origin_region': origin,
                'dest_region': dest,
                'origin_zip': region_to_zip[origin],
                'dest_zip': region_to_zip[dest],
                'flow': float(flow_value),
                'timestep': t,
                'datetime': timestep_to_datetime(t),
                'agent': agent_id
            })
            timestep_has_flow = True
        
        # If no flows for this timestep, add a placeholder row with tiny flow
        # This ensures the timestep isn't skipped in animation
        if not timestep_has_flow:
            origin_lon, origin_lat = region_centroids[0]
            dest_lon, dest_lat = region_centroids[0]  # Same point - invisible arc
            records.append({
                'origin_lon': origin_lon,
                'origin_lat': origin_lat,
                'dest_lon': dest_lon,
                'dest_lat': dest_lat,
                'origin_region': 0,
                'dest_region': 0,
                'origin_zip': region_to_zip[0],
                'dest_zip': region_to_zip[0],
                'flow': 0.001,  # Tiny flow - arc will be invisible
                'timestep': t,
                'datetime': timestep_to_datetime(t),
                'agent': agent_id
            })
    
    return pd.DataFrame(records)

# Create DataFrames for each agent (non-zero flows only)
df_agent0 = create_flow_df(flow_data, 0, region_centroids, flow_data['edges'])

## Alternative: Pre-configure Time Animation

If you want the time animation to be automatically enabled when the map loads, you can configure it programmatically:

In [6]:
# Configuration to enable time animation automatically
config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': 40.71,
            'longitude': -73.99,
            'zoom': 12.5,
            'pitch': 55,  # Camera tilt angle for 3D view (0-60)
            'bearing': -50,  # Camera rotation angle
            'dragRotate': True  # Allow 3D rotation
        },
        'visState': {
            'filters': [
                {
                    'dataId': ['Agent 0 Rebalancing Flows'],
                    'id': 'time-filter',
                    'name': ['datetime'],
                    'type': 'timeRange',
                    'value': [
                        df_agent0['datetime'].min(),
                        df_agent0['datetime'].max()
                    ],
                    'enlarged': True,
                    'plotType': 'histogram',
                    'animationWindow': 'free',  # 'free' shows all data points in time window
                    'speed': 1,  # Animation speed (1-10)
                    'yAxis': {
                        'type': 'value',
                        'domain': [0, 179]  # Time window in seconds (2 min 59 sec = 179 seconds)
                    }
                }
            ],
            'layers': [
                {
                    'type': 'geojson',
                    'config': {
                        'dataId': 'Manhattan Zip Code Areas',
                        'label': 'Manhattan Regions',
                        'color': [96, 96, 96],  # Light gray color
                        'columns': {
                            'geojson': '_geojson'
                        },
                        'isVisible': True,
                        'visConfig': {
                            'opacity': 0.3,
                            'strokeOpacity': 0.8,
                            'thickness': 1,
                            'strokeColor': [0, 0, 0],
                            'stroked': True,
                            'filled': True,
                            'enable3d': False,
                            'wireframe': True
                        }
                    }
                },
                {
                    'type': 'arc',
                    'config': {
                        'dataId': 'Agent 0 Rebalancing Flows',
                        'label': 'Rebalancing Arcs',
                        'color': [0, 0, 255],  # Blue for origin
                        'columns': {
                            'lat0': 'origin_lat',
                            'lng0': 'origin_lon',
                            'lat1': 'dest_lat',
                            'lng1': 'dest_lon'
                        },
                        'isVisible': True,
                        'visConfig': {
                            'opacity': 0.8,
                            'thickness': 2,
                            'colorRange': {
                                'name': 'Custom Blue to Yellow',
                                'type': 'sequential',
                                'category': 'Custom',
                                'colors': ['#0000FF', '#FFFF00']  # Blue to Yellow gradient
                            },
                            'sizeRange': [0, 10],  # Stroke width range
                            'targetColor': [255, 255, 0]  # Yellow for destination
                        },
                        'sizeField': {
                            'name': 'flow',
                            'type': 'real'
                        },
                        'sizeScale': 'linear'
                    },
                    'visualChannels': {
                        'sizeField': {
                            'name': 'flow',
                            'type': 'real'
                        },
                        'sizeScale': 'linear'
                    }
                }
            ]
        }
    }
}

# Create new map with time animation enabled
manhattan_animated = KeplerGl(height=1000, config=config)
manhattan_animated.add_data(data=manhattan_geojson, name='Manhattan Zip Code Areas')
manhattan_animated.add_data(data=df_agent0, name='Agent 0 Rebalancing Flows')
manhattan_animated

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 40.71, 'longitude': -73.99, 'zoom': 12.5…

In [7]:
# Export the animated map to HTML
manhattan_animated.save_to_html(file_name='manhattan_rebalancing_flows.html')

Map saved to manhattan_rebalancing_flows.html!
